In [6]:
import boto3
import dotenv, os

dotenv.load_dotenv("../.env")
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")

def upload_directory_to_s3(directory_path, bucket_name, s3_folder=""):
    """
    Uploads the contents of a directory to an S3 bucket.

    Args:
    directory_path (str): The path to the directory to upload.
    bucket_name (str): The name of the bucket to upload to.
    s3_folder (str): The folder path inside the S3 bucket to upload files into.

    Returns:
    bool: True if the directory was uploaded successfully, False otherwise.
    """
    s3_client = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)
    # Walk through each file in the directory
    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full S3 path
            relative_path = os.path.relpath(local_path, directory_path)
            s3_path = os.path.join(s3_folder, relative_path).replace("\\", "/")

            # Upload the file
            try:
                s3_client.upload_file(local_path, bucket_name, s3_path)
                print(f"Uploaded {local_path} to s3://{bucket_name}/{s3_path}")
            except Exception as e:
                print(f"Failed to upload {local_path}. Error: {str(e)}")
                return False
    return True

# Example usage: uploading 'my_directory' to 'your-bucket-name' under 'my_s3_folder/'
upload_directory_to_s3('transcripts', 'hackathon24s', 'test')


Uploaded transcripts/future_tech_clean.txt to s3://hackathon24s/test/future_tech_clean.txt
Uploaded transcripts/pinecone_clean.txt to s3://hackathon24s/test/pinecone_clean.txt
Uploaded transcripts/flexport_clean.txt to s3://hackathon24s/test/flexport_clean.txt
Uploaded transcripts/langchain.txt to s3://hackathon24s/test/langchain.txt
Uploaded transcripts/mongodb_clean.txt to s3://hackathon24s/test/mongodb_clean.txt
Uploaded transcripts/angellist_clean.txt to s3://hackathon24s/test/angellist_clean.txt
Uploaded transcripts/pinecone.txt to s3://hackathon24s/test/pinecone.txt
Uploaded transcripts/notion.txt to s3://hackathon24s/test/notion.txt
Uploaded transcripts/langchain_clean.txt to s3://hackathon24s/test/langchain_clean.txt
Uploaded transcripts/twitch.txt to s3://hackathon24s/test/twitch.txt
Uploaded transcripts/future_tech.txt to s3://hackathon24s/test/future_tech.txt
Uploaded transcripts/mongodb.txt to s3://hackathon24s/test/mongodb.txt
Uploaded transcripts/notion_clean.txt to s3://

True